In [1]:
import numpy as np
import pandas as pd
import time
import random

In [ ]:
 error = 2 * (output - lab_train) / output.shape[0] * self.softmax(params2['Z3'])
        change_w['W3'] = np.outer(error,params2['A2'])
        
        # Calculate b3 update
        #error = 2 * (output - lab_train) / output.shape[0] * self.softmax(params2['Z3'])
        #change_w['W3'] = np.outer(error,params2['A2'])
        
        # Calculate W2 update
        error = np.dot(params2['W3'].T, error) * self.sigmoidDelta(params2['Z2'])
        change_w['W2'] = np.outer(error, params2['A1'])

        # Calculate W1 update
        error = np.dot(params2['W2'].T, error) * self.sigmoidDelta(params2['Z1'])
        change_w['W1'] = np.outer(error, params2['A0'])
        
    
    
    # STEP 1: feedforward --------------------------------------------
                # z2 = W1.dot(x_i) + b1
                a1 = x_i
                #print(((params2["W1"]).dot(a1.T)).shape)
                z2 = params2["W1"].dot(a1.T) + params2["b1"]
                
                # a2 = sigmoid(z2)
                a2 = self.sigmoid(z2)
                
                # z3 = W2.dot(a1) + b2
                z3 = params2["W2"].dot(a2) + params2["b2"]
                
                # a3 = sigmoid(z3)
                a3 = self.sigmoid(z3)
                
                # End of feedforward --------------------------------------------
                
                # STEP 2: backpropagation ----------------------------------------
                delta3 = (a3 - y_i)  # error on last layer
        
                # gradients of last layer
                grad_w2 = (delta3.T).dot(a2.T) / batch_size
                grad_b2 = np.sum(delta3, axis=0) / batch_size
                
                # back propagate up to first layer
                delta2 = (delta3.dot(params2["W2"])).dot(self.sigmoidPrime(z2))
                grad_w1 = (delta2.T).dot(a1) / batch_size
                grad_b1 = np.sum(delta2, axis=0) / batch_size
  
                # regularization of backpropagation
                grad_w2 = grad_w2[:, None] + (self.lam * params2["W2"]) / training_size
                grad_w1 = grad_w1[:, None] + (self.lam * params2["W1"]) / training_size
            
                # STEP 3: update weights/ gradient decent -----------------------------------------
                params2["W1"] = params2["W1"] - self.alpha * grad_w1
                params2["b1"] = params2["b1"] - self.alpha * grad_b1
                params2["W2"] = params2["W2"] - self.alpha * grad_w2
                params2["b2"] = params2["b2"] - self.alpha * grad_b2

In [3]:
class myNeuralNet():
    
    def __init__(self, params1, epochs = 10, alpha = 0.9, beta = 0.9):
        self.params1 = params1
        self.epochs = epochs
        self.alpha = alpha
        self.beta = beta
        self.params2 = self.initialization()
    
    # initialize all the parameters for use in the neuralnet class
    def initialization(self):
        # number of nodes in each layer
        input_layer = self.params1[0]  # input layer has 784 nodes
        hidden_1 = self.params1[1]     # first hidden layer 128
        hidden_2 = self.params1[2]     # second hidden layer 64
        output_layer = self.params1[3] # output layer has 10 nodes

        params2 = {
            'W1'  : np.random.randn(hidden_1, input_layer) * np.sqrt(1. / hidden_1),
            'W2'  : np.random.randn(hidden_2, hidden_1) * np.sqrt(1. / hidden_2),
            'W3'  : np.random.randn(output_layer, hidden_2) * np.sqrt(1. / output_layer),
            'b1'  : np.zeros((hidden_1, 1)) * np.sqrt(1. / input_layer),
            'b2'  : np.zeros((hidden_2, 1)) * np.sqrt(1. / hidden_1),
            'b3'  : np.zeros((output_layer, 1)) * np.sqrt(hidden_2)
        }
        return params2
    
    # use sigmoid activagtion function
    def sigmoid(self, x):
        return 1.0/(1 + np.exp(-x))

    # obtain the derivative for backpropagating
    def sigmoidDelta(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))
    
    # use a deactivation function
    def softmax(self, x):
        exps = np.exp(x - x.max())
        return exps / np.sum(exps, axis=0)
     
    # get its derivative for back propagating
    def softmaxDelta(self, x):
        exps = np.exp(x - x.max())
        return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis=0))
    
    
    def forward_prop(self, x_train):
        
        params2 = self.params2
        
        # input layer as our training sample
        params2['A0'] = x_train

        # input layer to hidden layer 1
        params2['Z1'] = np.dot(params2["W1"], activations['A0']) #+ #params2['b1']
        params2['A1'] = self.sigmoid(activations['Z1'])

        # hidden layer 1 to hidden layer 2
        params2['Z2'] = np.dot(params2["W2"], activations['A1']) #+ #params2['b2']
        params2['A2'] = self.softmax(activations['Z2'])

        # hidden layer 2 to output layer
        params2['Z3'] = np.dot(params2["W3"], params2['A2']) #+ params2['b3']
        params2['A3'] = self.softmax(params2['Z3'])

        return params2['A2']
   
    # get updates to the changes of the output of the forward pass
    def backward_prop(self, y_train, activations, m_batch):
    
        params2 = self.params2

        # get the error from the last layer
        derivative_Z2 = activations["A2"] - y_train

        # get gradients get the last layer
        derivative_W2 = (1. / m_batch) * np.dot(derivative_Z2, activations["A1"].T)
        derivative_B2 = (1. / m_batch) * np.sum(derivative_Z2, axis = 1, keepdims = True)

        # then back propagate it to the first layer
        derivative_A1 = np.dot(params2["W2"].T, derivative_Z2)
        derivative_Z1 = derivative_A1 * self.sigmoidDelta(activations['Z1'])

        # get gradients at the first layer
        derivative_W1 = (1. / m_batch) * np.dot(derivative_Z1, x_train.T)
        derivative_B1 = (1. / m_batch) * np.sum(derivative_Z1, axis = 1, keepdims = True)

        gradients = {"DW1" : derivative_W1, "DB1" : derivative_B1, "DW2" : derivative_W2, "DB2" : derivative_B2}

        return gradients  

    # compute the accuracy of the validation tests
    def calculate_accuracy(self, x_valid, y_valid):
        validations = []
        
        for x, y in zip(x_valid, y_valid):
            
            output = self.forward_pass(x)
            validate = np.argmax(output)
            validations.append(validate == np.argmax(y))
            #pd.DataFrame(validate).to_csv('test_predictions.csv',index = False)
        return np.mean(validations)

    def train(self, x_train, y_train, x_valid, y_valid):
        
        params2 = self.params2
        start_time = time.time() 
        
        # initialize for momentum to keep a moving 
        # average of our gradients
        grad_w1 = np.zeros(params2["W1"].shape)
        grad_b1 = np.zeros(params2["b1"].shape)
        grad_w2 = np.zeros(params2["W2"].shape)
        grad_b2 = np.zeros(params2["b2"].shape)
        grad_w3 = np.zeros(params2["W3"].shape)
        grad_b3 = np.zeros(params2["b2"].shape)

        for iteration in range(self.epochs):
            
            
            for x_i, y_i in zip(x_train, y_train):
            
                # STEP 1: feedforward --------------------------------------------
                activations = self.forward_prop(x_i, params2)
                
                # STEP 2: backpropagation ----------------------------------------
                gradients = self.backward_prop(y_i, activations)
                
                # Momentum step ----------------------------------------
                grad_w1 = (self.beta * grad_w1 + (1. - self.beta) * gradients['W1'])
                grad_w2 = (self.beta * grad_w2 + (1. - self.beta) * gradients['W2'])
                grad_w3 = (self.beta * grad_w3 + (1. - self.beta) * gradients['W3'])
                
                #grad_b1 = (self.beta * grad_b1 + (1. - self.beta) * gradients['b1'])
                #grad_b2 = (self.beta * grad_b2 + (1. - self.beta) * gradients['b2'])
                #grad_b3 = (self.beta * grad_b3 + (1. - self.beta) * gradients['b3'])
                
                # STEP 3: update weights/ gradient decent -----------------------------------------
                params2["W1"] = params2["W1"] - self.alpha * grad_w1
                params2["W2"] = params2["W2"] - self.alpha * grad_w2
                params2["W3"] = params2["W3"] - self.alpha * grad_w3
        
        accuracy = self.calculate_accuracy(x_valid, y_valid)
        print('Epoch: {0}, Time Spent: {1:.2f}s, Accuracy: {2:.2f}%'.format(
            iteration + 1, time.time() - start_time, accuracy * 100))
            

In [4]:
# load the data
images = pd.read_csv("mnist_image.csv", header = None)
labels = pd.read_csv("mnist_label.csv", header = None)

FileNotFoundError: [Errno 2] No such file or directory: 'mnist_image.csv'

In [285]:
# convert to arrays
images = np.asarray(images)
labels = np.asarray(labels)

In [286]:
#normalize image data
images = np.multiply(images, 1.0 / 255.0)
images = images.astype('float32')

In [287]:
# Do one-hot encoding for our labels
# e.g 7 => [0 0 0 0 0 0 0 1 0 0]
digits = 10
examples = labels.shape[0]
labels = labels.reshape(1, examples)
labels_new = np.eye(digits)[labels.astype('int32')]
labels_new = labels_new.T.reshape(digits, examples)

In [288]:
# split data into training and validation
size_train = 60000
size_test = images.shape[0] - size_train
images_train, images_valid = images[:size_train], images[size_train:]
labels_train, labels_valid = labels_new[:,:size_train], labels_new[:, size_train:]
labels_train = labels_train.T
labels_valid = labels_valid.T

In [1]:
start_time = time.time()
np.random.seed(138)
neuralnet = myNeuralNet(sizes=[784, 128, 64, 10]) # params = [input_size, hidden_1, hidden_2, output_size]
neuralnet.train(images_train, labels_train, images_valid, labels_valid)
print("--- %s seconds ---" % (time.time() - start_time))

NameError: name 'time' is not defined